In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
  !pip install wandb -qU
  from google.colab import runtime
  from google.colab import drive
  drive.mount('/content/drive')
  %cd '/content/drive/MyDrive/GitHub/in_dev_RN20Q'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.7/214.7 kB 24.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00
Mounted at /content/drive
/content/drive/MyDrive/GitHub/in_dev_RN20Q


In [ ]:
import wandb
import torch
from model.resnet4c10q import ResNet20_Q
from model.resnet4c10 import resnet20
from mylib.KDResNetTrainer import ResNetTrainer

wandb.login(key='e0c11d3ff2bee4c8775ba05863038fdac671c043')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
api = wandb.Api()
artifact = api.artifact('jke1994/ResNet20/model:v179')
artifact.download(root='./pretrained_model')


wandb:   1 of 1 files downloaded.  


'./pretrained_model'

In [ ]:
state_dict = torch.load('./pretrained_model/ckpt.pth', map_location=torch.device('cpu'))

# update all keys in state_dict to remove module.
state_dict['net'] = {k.replace('module.', ''): v for k, v in state_dict['net'].items()}

In [ ]:
teacher_model = resnet20()
# load pretrained model for resnet20
teacher_model.load_state_dict(state_dict['net'])

<All keys matched successfully>

In [ ]:
lr = 0.01
arch_tag = "ResNet20Q"
batch_size = 128
dataset = "CIFAR10"

student_model = ResNet20_Q(a_bit=4, w_bit=1)

run = wandb.init(
    # set the wandb project where this run will be logged
    project="ResNet20_Q",

    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": arch_tag,
    "batch_size": batch_size,
    "dataset": dataset
    }
)

my_trainer = ResNetTrainer(dataset=dataset,
                           arch_tag=arch_tag,
                           student_model=student_model,
                           teacher_model=teacher_model,
                           device = 'cuda',
                           batch_size=batch_size)

my_trainer.train_script(200)

wandb.finish()

runtime.unassign()

wandb: Currently logged in as: jke1994. Use `wandb login --relogin` to force relogin


Files already downloaded and verified
Files already downloaded and verified

Epoch: 0


Epoch 0:   0%|          | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:2919: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(
Epoch 0: 100%|██████████| 391/391 [00:21<00:00, 18.54it/s, train_acc=0.197, xentropy=1.468]
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:437: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


test_acc: 0.209
Saving..

Epoch: 1


Epoch 1: 100%|██████████| 391/391 [00:18<00:00, 20.76it/s, train_acc=0.238, xentropy=1.371]


...

Epoch 198: 100%|██████████| 391/391 [00:18<00:00, 20.85it/s, train_acc=0.684, xentropy=0.681]


test_acc: 0.627

Epoch: 199


Epoch 199: 100%|██████████| 391/391 [00:18<00:00, 21.23it/s, train_acc=0.682, xentropy=0.681]


test_acc: 0.656


cross_entropy,██▆▄▆▂▃▃▃▂▂▁▁▄▁▂▁▂▂▁▄▃▄▂▂▂▂▂▂▁▂▂▁▁▁▂▂▂▂▁
lr,████████▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁▃▂▂▄▂▄▃▇▇▇▆▄▆▇▃██▇▇▇▇▇▇█▇███▇▆▇█▇██▇▆▇▇
test_loss,▆▄▇█▅▇▇▆▂▂▂▃▆▃▂▇▁▁▂▂▂▂▂▂▁▂▁▁▁▂▃▂▁▂▁▂▂▃▂▁
train_acc,▁▃▄▅▆▆▆▆▇▇▇▇▇▇▇▇████████████████████████
train_loss,█▆▅▄▃▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
cross_entropy,0.82069
lr,1e-05
test_acc,0.6557
test_loss,0.00841
train_acc,0.6817
